# Emotion Chat Bot

In [1]:
import os.path
import uuid

import torch
from libs import (
	EmotionModel,
	EmotionPresentationSimilarityAnalyser,
	ResponseGeneratorPipeline,
	generate_dummy_representation,
	get_sentiment_composition,
)
from libs.FullModel import create_candidates_buffer, get_top_emotion, write_log_file
from sympy.core.random import randint
from torch import Tensor
from transformers import (
	AutoModelForSequenceClassification,
	AutoTokenizer,
	BitsAndBytesConfig,
	TextStreamer,
	pipeline,
)
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## Load Each Module

### Response Generator

In [2]:
model, tokenizer = FastLanguageModel.from_pretrained(
	model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
	attn_implementation="flash_attention_2",
	pretraining_tp=1,
	load_in_4bit=True,
	device_map="auto",
	low_cpu_mem_usage=True,
	trust_remote_code=True,
)
tokenizer.padding_side = "left"
tokenizer.clean_up_tokenization_spaces = True

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


In [3]:
response_generator = ResponseGeneratorPipeline(
	model,
	tokenizer,
	framework="pt",
	task="conversation-generation",
	num_workers=16,
	torch_dtype="auto",
	add_special_tokens=True,
	truncation=False,
	padding=True,
)

In [4]:
FastLanguageModel.for_inference(response_generator.model)
response_generator.model = torch.compile(response_generator.model)

### Sentiment Analyzer

In [5]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
	"Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
	quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16),
	device_map="auto",
	low_cpu_mem_usage=True,
)

In [6]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
	"Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
	trust_remote_code=True,
)

In [7]:
sentiment_analyzer = pipeline(
	"sentiment-analysis",
	model=sentiment_analysis_model,
	tokenizer=sentiment_analysis_tokenizer,
	top_k=7,
	torch_dtype=torch.float32,
	device_map="auto",
	trust_remote_code=True,
)

In [8]:
sentiment_analyzer.model = torch.compile(sentiment_analyzer.model)

In [9]:
print(sentiment_analyzer.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Predictor

In [10]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
	"Shotaro30678/emotion_predictor_for_emotion_chat_bot",
	quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16),
	device_map="auto",
	low_cpu_mem_usage=True,
)

In [11]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
	"Shotaro30678/emotion_predictor_for_emotion_chat_bot",
	trust_remote_code=True,
)

In [12]:
emotion_predictor = pipeline(
	"sentiment-analysis",
	model=emotion_predictor_model,
	tokenizer=emotion_predictor_tokenizer,
	top_k=7,
	torch_dtype=torch.float32,
	device_map="auto",
	trust_remote_code=True,
)

In [13]:
emotion_predictor.model = torch.compile(emotion_predictor.model)

In [14]:
print(emotion_predictor.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Model

In [15]:
emotion_model = EmotionModel.from_pretrained("hermeschen1116/emotion_model_for_emotion_chat_bot")

In [16]:
emotion_model = torch.compile(emotion_model, mode="reduce-overhead")

In [17]:
print(emotion_model)

OptimizedModule(
  (_orig_mod): EmotionModel(
    (_EmotionModel__weight_Q): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__weight_K): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__dropout): Dropout(p=0.5, inplace=False)
    (_EmotionModel__weight_D): Linear(in_features=7, out_features=7, bias=True)
  )
)


### Similarity Analyzer

In [18]:
threshold: float = 0.5

In [19]:
similarity_analyzer = EmotionPresentationSimilarityAnalyser(None, threshold=threshold)

## Combine All Modules

In [29]:
emotions: list = [
		"neutral",
		"anger",
		"disgust",
		"fear",
		"happiness",
		"sadness",
		"surprise",
	]

### Initialize

In [30]:
chat_buffer_size: int = 10

In [31]:
logfile_uuid: uuid.UUID = uuid.uuid4()
if not os.path.isdir("./logs"):
	os.mkdir("./logs")

In [32]:
streamer = TextStreamer(tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [34]:
bot_ideal_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
similarity_analyzer.ideal_emotion_representation = bot_ideal_emotion_representation
dict(zip(emotions, bot_ideal_emotion_representation.tolist()))

{'neutral': 0.3390445113182068,
 'anger': 0.9162819385528564,
 'disgust': 0.42804205417633057,
 'fear': 0.5070522427558899,
 'happiness': 0.448799192905426,
 'sadness': 0.16032946109771729,
 'surprise': 0.8833631873130798}

In [36]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

tensor([0.0489, 0.1856, 0.7903, 0.6453, 0.4324, 0.6348, 0.7582])

In [37]:
system_prompt: str = input("Enter your system prompt: ").strip()
bot_response: str = "Talk to me..."

In [38]:
chat_buffer: list = [{"role": "system", "content": {"emotion": "", "dialog": system_prompt}}]

In [39]:
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}}]

In [40]:
while True:
	user_response: str = input(f"Bot: {bot_response}").strip()
	if user_response == "quit":
		break

	user_emotion: list = sentiment_analyzer(user_response)[0]
	chat_buffer.append(
		{
			"role": "user",
			"content": {
				"emotion": get_top_emotion(user_emotion),
				"dialog": user_response,
			},
		}
	)

	user_emotion_composition: Tensor = get_sentiment_composition(user_emotion)

	bot_emotion_representation = emotion_model.forward(user_emotion_composition, bot_emotion_representation)

	bot_chat_simulations: list = []
	while True:
		bot_chat_simulations = create_candidates_buffer(chat_buffer)
		bot_chat_simulations = [chat[0] for chat in response_generator(bot_chat_simulations, streamer=streamer)]
		print()
		write_log_file(logfile_uuid, bot_chat_simulations)
		bot_chat_simulations = list(filter(lambda chat: chat[-1]["content"]["dialog"] != "", bot_chat_simulations))
		if len(bot_chat_simulations) != 0:
			break

	bot_response_simulations: list = [chat[-1]["content"] for chat in bot_chat_simulations]

	user_future_emotion_composition_simulations: dict = {
		response["emotion"]: get_sentiment_composition(emotion_predictor(response["dialog"])[0])
		for response in bot_response_simulations
	}

	bot_future_emotion_representations: dict = {
		k: emotion_model.forward(v, bot_emotion_representation)
		for k, v in user_future_emotion_composition_simulations.items()
	}

	emotion_representation_similarity_scores: list = similarity_analyzer(
		torch.stack(list(bot_future_emotion_representations.values())),
	)

	candidate_emotion_index: int = torch.argmax(emotion_representation_similarity_scores).item()
	bot_best_response_emotion: str = emotions[candidate_emotion_index]

	if len(chat_buffer) == chat_buffer_size + 1:
		chat_buffer.pop(1)
	chat_buffer.append({"role": "bot", "content": {"emotion": bot_best_response_emotion, "dialog": ""}})

	chat_buffer = response_generator(chat_buffer, streamer=streamer)[0]
	print()
	if chat_buffer[-1]["content"]["dialog"] == "":
		chat_buffer[-1]["content"]["dialog"] = bot_response_simulations[candidate_emotion_index]["dialog"]
	bot_response = chat_buffer[-1]["content"]["dialog"]
	write_log_file(logfile_uuid, chat_buffer)

TypeError: 'str' object is not callable